<a href="https://colab.research.google.com/github/syKevinPeng/ExperimentResultAnalysis/blob/main/result_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# process_result.py
from google.colab import drive
import os
import pathlib

# mount google drive
drive.mount('/content/drive')

# path to directory
result_dir = "/content/drive/MyDrive/Research/VRExperiment"

# List all files in the folder
file_list = os.listdir(result_dir)
for file in file_list:
  print(file)